### Because the output data is huge, we should take the vertical/zonal average in a separate step and save results for further analysis and plotting.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cubedsphere as cs

## Vertical mean

In [4]:
# warning: this cell takes very long to run!
# the highest-res case (C384L160) might sometimes fail

Vres_list = [20,40,80,160]#[3:]
Hres_list = [48,96,192,384]#[3:]

outputdir = "~/FV3/output/processed/vertical_mean/"

for i,Hres in enumerate(Hres_list):
    for j,Vres in enumerate(Vres_list):
        
        print('loading',Hres,Vres,end=' ')
        datadir = ("../../output/C{0}/C{0}L{1}/".format(Hres,Vres))
        ds = cs.open_FV3data(datadir,"tracer_daily")
        
        # only load the last day
        dr = ds['plume01'].isel(time=-1).copy(); dr.load()
        
        print('calculating...',end=' ')
        dr_vmean = dr.mean(dim='pfull')
        ds_vmean = dr_vmean.to_dataset()
        
        # recover lost coordinates
        ds_vmean['lat_b'] = ds['lat_b']
        ds_vmean.set_coords(["lat_b"], inplace=True)
        ds_vmean['lon_b'] = ds['lon_b']
        ds_vmean.set_coords(["lon_b"], inplace=True)
        
        print('writting...',end='\n')
        ds_vmean.to_netcdf(outputdir+'plume01_vmean_C{0}L{1}.nc'.format(Hres,Vres))

loading 384 160 calculating... writting...


## Meridional mean

In [3]:
# warning: this cell takes very long to run!

Vres_list = [20,40,80,160]#[3:]
Hres_list = [48,96,192,384]#[3:]

outputdir = "~/FV3/output/processed/meri_mean/"

for i,Hres in enumerate(Hres_list):
    for j,Vres in enumerate(Vres_list):
        
        print('loading',Hres,Vres,end=' ')
        datadir = ("../../output/C{0}/C{0}L{1}/".format(Hres,Vres))
        ds = cs.open_FV3data(datadir,"tracer_daily")
        
        # only load the last day
        dr = ds['plume01'].isel(time=-1).copy(); dr.load()
        
        print('calculating...',end=' ')
        # use finer bin at higher Hres
        dr_meri = cs.meridional_mean(dr,dlon=10.0/Hres*48)
        
        print('writting...',end='\n')
        dr_meri.to_netcdf(outputdir+'plume01_meri_C{0}L{1}.nc'.format(Hres,Vres))

loading 384 160 calculating... writting...


## Whole time series of the last day

In [3]:
Vres = 160
Hres = 384

outputdir = "~/FV3/output/processed/timeseries/"

print('loading',Hres,Vres,end=' ')
datadir = ("../../output/C{0}/C{0}L{1}/".format(Hres,Vres))
ds = cs.open_FV3data(datadir,"tracer_daily")

dr = ds['plume01']
dr

loading 384 160 

<xarray.DataArray 'plume01' (tile: 6, time: 8, pfull: 160, y: 384, x: 384)>
dask.array<concatenate, shape=(6, 8, 160, 384, 384), dtype=float64, chunksize=(1, 8, 160, 384, 384)>
Coordinates:
  * x        (x) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * y        (y) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * pfull    (pfull) float64 2.205 6.781 13.27 19.62 25.92 32.21 38.49 44.76 ...
  * time     (time) float64 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0
    lon      (tile, y, x) float32 305.097 305.293 305.488 305.685 305.881 ...
    lat      (tile, y, x) float32 -35.2184 -35.3098 -35.4009 -35.4917 ...
    area     (tile, y, x) float32 3.6204e+08 3.62862e+08 3.63679e+08 ...
Dimensions without coordinates: tile
Attributes:
    long_name:     plume01
    units:         none
    cell_methods:  time: point

In [4]:
# vertical mean

print('calculating...',end=' ')
dr_vmean = dr.mean(dim='pfull')
ds_vmean = dr_vmean.to_dataset()

# recover lost coordinates
ds_vmean['lat_b'] = ds['lat_b']
ds_vmean.set_coords(["lat_b"], inplace=True)
ds_vmean['lon_b'] = ds['lon_b']
ds_vmean.set_coords(["lon_b"], inplace=True)

print('writting...',end='\n')
ds_vmean.to_netcdf(outputdir+'plume01_ts_vmean_C{0}L{1}.nc'.format(Hres,Vres))

calculating... writting...


In [5]:
# meridional mean

print('calculating...',end=' ')
# use finer bin at higher Hres
dr_meri = cs.meridional_mean(dr,dlon=10.0/Hres*48)

print('writting...',end='\n')
dr_meri.to_netcdf(outputdir+'plume01_ts_meri_C{0}L{1}.nc'.format(Hres,Vres))

calculating... writting...
